In [3]:
from openai import OpenAI
import json
import chromadb

# create an OpenAI client
client = OpenAI()

#create a chromaDB client
chroma_client = chromadb.PersistentClient()
collection = chroma_client.get_or_create_collection(name="Real_Estate" , metadata={"hnsw:space": "cosine"} )

# Check if there is any existing data in the collection
collection.count()


38

In [6]:
questions = [   
                "How big do you want your house to be?" ,
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]

In [9]:
answers = []
for i,question in enumerate(questions):
    answers.append(input(question + "\n"))


In [42]:
results = collection.query(
    query_texts=answers, # Chroma will embed this for you
    n_results=3 # how many results to return
)

In [43]:
House_list = [item.split("_")[0] for sublist in results["ids"] for item in sublist]

In [44]:
Unique_House_list = list(set(House_list))

In [48]:
Unique_House_list


['389cb206-96ce-424e-897f-9dc22bdb47f9',
 '0df9767f-6885-49da-a533-fa94abdf4959',
 'af920a74-3b52-4536-9225-4c986c64af4d',
 '25b66d8e-afc8-4e5c-9842-2535e93df695',
 '2e65b8f7-abb7-40d7-94ce-7137a0ff3f53',
 'fc43b2d2-9a04-4f3a-bd63-4765ab7c7b25',
 '799b525d-797c-4c1c-9f30-0ecd3a50889a',
 '99a010cc-77de-4bed-b21c-4e359b660c42',
 'fdc1b68b-c524-489b-a2eb-064ab268602e']

In [73]:
print(f"Here are the {len(Unique_House_list)} houses that match your preferences:")

for i,id in enumerate(Unique_House_list):
    print(f"House {i+1}: \n")
    Des_data = collection.get(ids=[f"{id}_Description"])
    #print(Des_data)
    Description = Des_data['documents'][0]
    Nei_Des_data = collection.get(ids=[f"{id}_Neighborhood_Description"])
    Neighborhood_Description = Nei_Des_data['documents'][0]
    #print(Description,Neighborhood_Description)

    Userpreferences = " "
    for j,question in enumerate(questions):
        Userpreferences = Userpreferences + "Question:" + questions[j] + "User Preference:" + answers[j] + "\n "

    Example = "Comfortable house with a spacious kitchen and a cozy living room in the charming Ravenswood neighborhood. This area is known for its tree-lined streets, historic architecture, and welcoming community vibe. You will find good local schools, convenient shopping options, and a quiet neighborhood in Ravenswood. Additionally, with easy access to the Metra station and public transportation, commuting to downtown Chicago will be a breeze. Look for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system to suit your needs"
    messages = [{"role": "user", "content": f"Given the information of House Description as {Description} and Neighborhood Description as {Neighborhood_Description} , User's preference is: {Userpreferences}  provide a personalized message for the user. Example: {Example}"}]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    response_message = response.choices[0].message.content
    print(response_message)






Here are the 9 houses that match your preferences:
House 1: 

Comfortable three-bedroom house with a spacious kitchen and cozy living room in the heart of Maple Grove. This charming neighborhood is perfect for families, with tree-lined streets, excellent schools, and convenient shopping options just a short drive away. Enjoy the tranquil surroundings and upscale amenities in this Maple Grove masterpiece. Look for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system to suit your needs. With a strong sense of community and a variety of recreational activities, Maple Grove offers the perfect blend of suburban tranquility and urban convenience for you.
House 2: 

Your ideal home awaits in the peaceful neighborhood of Golden Meadows. Imagine coming home to a comfortable three-bedroom house with a spacious kitchen and cozy living room, perfect for unwinding after a long day. With access to top-rated local schools, convenient shopping optio